In [ ]:
import contextily as cx
import pandas as pd
import geopandas

## Loading data

In [ ]:
'data-ambassadors-parking-project/data/raw/friedrichshain-kreuzberg/lor_friedrichshain_kreuzberg.GeoJSON'

In [ ]:
lor_gdf = geopandas.read_file("data-ambassadors-parking-project/data/raw/friedrichshain-kreuzberg/lor_friedrichshain_kreuzberg.GeoJSON")
streets_processed_gdf = geopandas.read_file('data-ambassadors-parking-project/data/datathon/extras/streets_processed.geojson')
drive_ways_gdf = geopandas.read_file("data-ambassadors-parking-project/data/datathon/extras/drive_ways.geojson")
service_processed_gdf = geopandas.read_file("data-ambassadors-parking-project/data/datathon/extras/service_processed.geojson")
estimated_parking_lanes_gdf = geopandas.read_file("data-ambassadors-parking-project/data/datathon/parking/estimated_parking_lanes.geojson")
estimated_parking_spots_kfz_gdf = geopandas.read_file("data-ambassadors-parking-project/data/datathon/parking/estimated_parking_spots_kfz.geojson")
demographics_age_df = pd.read_csv("data-ambassadors-parking-project/data/datathon/demographics/demographics_age_of_inhabitants.csv", thousands=" ")
demographics_background_df = pd.read_csv("data-ambassadors-parking-project/data/datathon/demographics/demographics_background_of_inhabitants.csv", thousands=" ", decimal=",")
registered_cars_gdf = geopandas.read_file("data-ambassadors-parking-project/data/datathon/demographics/registered_cars.geojson")

# My data import
cartagger_gdf = geopandas.read_file('data-ambassadors-parking-project/data/datathon/parking/counted_cars_from_cartagger_project.geojson')


## Merging some dataframes

In [ ]:
demographics_df = pd.merge(demographics_age_df, demographics_background_df, on=("lor_number", "lor"))

In [ ]:
stats_df = pd.merge(demographics_df, registered_cars_gdf, on="lor")

## Visualizing data

In [ ]:
def get_lor_map():
    return lor_gdf.explore(
        column="PLR_NAME",
        legend=True,
        legend_kwds={"colorbar": False},
        style_kwds={"opacity": 0.4, "fillOpacity": 0.4},
        name="Friedrichshain-Kreuzberg"
    )

In [ ]:
def plot_geodataframe(gdf: geopandas.GeoDataFrame):
    ax = gdf.plot(figsize=(20, 20), alpha=0.5, edgecolor="k")
    cx.add_basemap(ax, crs=gdf.crs)

In [ ]:
get_lor_map()

# Read POI data

In [ ]:
osm = pyrosm.OSM(input_filepath)
network = osm.get_network(
    network_type="cycling",
    extra_attributes=["surface", "smoothness"]
)

# Join Cartagger polygon to parking Spot point

In [ ]:
spots_filled_gdf1 = geopandas.sjoin(cartagger_gdf, estimated_parking_spots_kfz_gdf, how="left", op="contains")
spots_filled_gdf2 = geopandas.sjoin(estimated_parking_spots_kfz_gdf, cartagger_gdf, how="left", op="within")

spots_filled_gdf2.head()

# plot car tagger and spot data

In [ ]:
cartagger_gdf.explore(m=estimated_parking_spots_kfz_gdf.explore())

In [ ]:
spots_filled_gdf = geopandas.sjoin(cartagger_gdf, estimated_parking_spots_kfz_gdf, how="inner", predicate='overlaps')
spots_filled_gdf

# Michel's Function

In [ ]:
# Imports
import osmnx as ox
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


def get_osm_data(lor_gdf, tags, crs="EPSG:25833", location="Berlin, Germany"):
    # Load data
    try:
        lor_gdf = gpd.read_file(lor_gdf)
    except:
        print("Can't load LOR file")
    # Obtain coordinates of POIs
    poi = ox.geometries_from_place(location, tags=tags)
    poi_points = poi[poi.geom_type == "Point"]

    # Drop unnecessary (debatable) information
    poi = poi_points.dropna(axis=1)
    poi["name"] = poi_points["name"]

    # Project to the same coordinate system
    poi = poi.to_crs(crs)

    # We look for the intersection of all pois and our Bezirk
    poi_lor = gpd.sjoin(poi, lor_gdf, predicate="intersects")

    return poi_lor


def calc_parking_spots(poi_lor, cars, est_parking_spots, buffer_size=20):
    # Load data
    try:
        est_parking_spots = gpd.read_file(est_parking_spots)
    except:
        print("Can't load parking spots file")

    # Define a circular region around each poi
    poi_lor_buffered = poi_lor.copy()
    poi_lor_buffered["geometry"] = poi_lor["geometry"].buffer(buffer_size)
    poi_lor_buffered = poi_lor_buffered.drop(labels=["index_right"], axis=1)

    # Intersect parking spots and area
    intersect_parking = gpd.sjoin(poi_lor_buffered, est_parking_spots)
    # Intersect tagged cars and area
    intersect_cars = gpd.sjoin(poi_lor_buffered, cars)
    

    # Calculate parking spots
    intersect_parking["capacity"] = intersect_parking["capacity"].apply(int)
    dfpivot = pd.pivot_table(
        intersect_parking,
        index="name_left",
        columns="capacity",
        aggfunc={"capacity": len},
    )
    dfpivot.columns = dfpivot.columns.droplevel()
    dfpivot.index.names = ["name"]
    parking_counts = dfpivot.sum(axis=1)
    parking_counts = parking_counts.to_frame("No. parking spots")
    
    # Calculate tagged cars
    intersect_cars["capacity"] = intersect_cars["capacity"].apply(int)
    dfpivot = pd.pivot_table(
        intersect_cars,
        index="name_left",
        columns="capacity",
        aggfunc={"capacity": len},
    )
    dfpivot.columns = dfpivot.columns.droplevel()
    dfpivot.index.names = ["name"]
    car_counts = dfpivot.sum(axis=1)
    car_counts = parking_counts.to_frame("No. cars tagged")

    # Merge back with the main dataframe
    poi_lor_counts = poi_lor.merge(parking_counts.reset_index(), on="name", how="left")
    poi_lor_counts["No. parking spots"] = poi_lor_counts["No. parking spots"].fillna(0)
    poi_lor_counts["No. cars tagged"] = poi_lor_counts["No. parking spots"].fillna(0)
    
    #poi_lor_buffered["No. parking spots"] = poi_lor_counts["No. parking spots"].fillna(0)

    return poi_lor_counts, poi_lor_buffered

In [ ]:
poi_lor_buffered = poi_lor.copy()
poi_lor_buffered["geometry"] = poi_lor["geometry"].buffer(40)
poi_lor_buffered = poi_lor_buffered.drop(labels=["index_right"], axis=1)

gpd.sjoin(poi_lor_buffered, cartagger_gdf).head()

In [ ]:
# Usage:
poi_lor = get_osm_data(
    r"data-ambassadors-parking-project/data/raw/friedrichshain-kreuzberg"+"/lor_friedrichshain_kreuzberg.GeoJSON",
    {"amenity": True, "shops":True},
    )

In [ ]:
parking_counts, parking_counts_buffered = calc_parking_spots(
    poi_lor, cartagger_gdf,
    r"data-ambassadors-parking-project/data/datathon"+"/parking/estimated_parking_spots_kfz.geojson",
    40,
    )
#parking_counts_buffered.explore(m=estimated_parking_spots_kfz_gdf.explore(m=cartagger_gdf.explore()))

In [ ]:
parking_counts.head()

In [ ]:
parking_counts_buffered.head()